# Loading Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/mushroom-classification/mushrooms.csv')

# Exploring Data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['habitat'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
def label_encoded(feat):
    le = LabelEncoder()
    le.fit(feat)
    print(feat.name,le.classes_)
#     print(le.classes_)
    return le.transform(feat)

In [ ]:
for col in df.columns:
    df[str(col)] = label_encoded(df[str(col)])

In [ ]:
df.head()

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
plt.figure(figsize=(12,10))
ax = sns.heatmap(df.corr())

In [ ]:
fig = plt.figure(figsize = (20,15))
ax = fig.gca()
df.hist(ax=ax)
plt.show()

# Model 1 remove less corr. data  

In [ ]:
X = df.drop(['class','veil-type','gill-attachment','ring-type','gill-color','bruises'],axis=1)
y = df['class']

## Split Data

In [ ]:
 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4,random_state=42)

### Building model 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_1 = RandomForestClassifier(max_depth=10, random_state=10)
model_1.fit(X_train, y_train)


In [ ]:
y_pred = model_1.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

#  Model 2 Using All Features

In [ ]:
X = df.drop(['class'],axis=1)
y = df['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4,random_state=42)

In [ ]:
model_2 = RandomForestClassifier(max_depth=10, random_state=0)
model_2.fit(X_train, y_train)

In [ ]:
y_pred = model_2.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

# Model 3 Using NN 

In [ ]:
X = df.drop(['class','veil-type','gill-attachment','ring-type','gill-color','bruises'],axis=1)
y = df['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33,random_state=42)

In [ ]:
df['class'].value_counts()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model_NN = Sequential()
model_NN.add(Dense(32, input_dim=17, activation='tanh'))
# Dropout(0.2),
model_NN.add(Dense(16, activation='tanh'))
# Dropout(0.2),
model_NN.add(Dense(1, activation='softmax'))
# Compile model
model_NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10,restore_best_weights=True)

In [ ]:
model_NN.fit(x=X_train, 
          y=y_train, 
          epochs=500,
#           batch_size=200,
          validation_data=(X_test, y_test),
           callbacks=[early_stop]
          )

In [ ]:
model_NN.evaluate(X_test, y_test)

In [ ]:
pred = model_NN.predict(X_test)
y_test.iloc[9]

In [ ]:
model_NN.predict(np.array([X_test.iloc[9]]))
